In [1]:
import pandas as pd
import numpy as np
import pyarrow
import fastparquet


import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,lpad, length, lit, to_date,to_timestamp, when, date_format, coalesce, md5, sha2, concat_ws,trim
from pyspark.sql import functions as F

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import datetime
from pyspark.sql.functions import round
from pyspark.sql.functions import year
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import year
from pyspark.sql.functions import col, sum, min, max
from pyspark.sql.functions import first


In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Qlik_transfo") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/10 09:13:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path_ = "/Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/datasetV1.xlsx"
file_path = "/Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/Frasi_Gestion.xlsx"
Suivi_et_Archive = "Suivi_et_Archive"
Tab_PARAM = "Tab_PARAM"
PARAM_FG = "PARAM_FG"

# files to dataframes

df_Suivi_et_Archive = pd.read_excel(file_path_, header=1)
df_Tab_PARAM = pd.read_excel(file_path, Tab_PARAM)
df_PARAM_FG = pd.read_excel(file_path, PARAM_FG)

# effecaer les colonnes qui sont vides
columns_to_delete = ['Unnamed: 9', 'Unnamed: 12',"Unnamed: 16",]
# Effacer les colonnes vide
df_PARAM_FG = df_PARAM_FG.drop(columns=columns_to_delete)

# schema de Param FG

schema_PARAM_FG = StructType([
    StructField('Unnamed: 0', StringType(), True),
    StructField('Année', StringType(), True),
    StructField('Catégorie', StringType(), True),
    StructField('Sous-Catégorie',StringType(), True),
    StructField('Recueillir les candidatures TTC', IntegerType(), True),
    StructField('Instruire les dossiers TTC',IntegerType(), True),
    StructField('Accompagner les projets TTC', IntegerType(), True),
    StructField('Recueillir les candidatures HT',IntegerType(), True),
    StructField('Instruire les dossiers HT', IntegerType(), True),
    StructField('Accompagner les projets HT',IntegerType(), True),
    StructField('Reporting et évaluation TTC', IntegerType(), True),
    StructField('Reporting et évaluation HT',IntegerType(), True),
    StructField('Convention SCAN', StringType(), True),
    StructField('Convention', StringType(), True),
    StructField('Sous-fonds FG AAP/Projets',StringType(), True),
    StructField('Dispositif', StringType(), True),
    StructField('Dispositif Cible', StringType(), True),
    StructField('AAP', StringType(), True),
    StructField('Date de relève',StringType(), True),
    StructField('Acronyme du projet', StringType(), True),
    StructField("Nom de l'entreprise", StringType(), True),
    StructField("Identifiant projet",StringType(), True),
    StructField("Montant total de l'aide retenue HT", IntegerType(), True),
    StructField('COPIL : Date de signature DPM',StringType(), True),
    StructField('Convention.1', StringType(), True),
    StructField('AAP_Gré-à-gré', StringType(), True),])

# MAPPING
PARAM_FG = spark.read.csv('df_PARAM_FG_clean.csv', header=True, schema=schema_PARAM_FG)

# Replace NaN values with None
PARAM_FG = PARAM_FG.na.replace(float('nan'), None)

# lire le dataframe principale 

df=spark.read.parquet("/Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/side_231218113615.parquet")


# table de ara
Param = df_Tab_PARAM[['Statut', "Terminé/En cours",'Liste_Relèves',"Stade_Relèves_FG"]]

Param = spark.createDataFrame(Param)
# rename stat to be able to map terminé/en cours properly
Param = Param.withColumnRenamed("Statut", "Statut_")

# Specify the path to your Excel file
excel_file_path = "/Users/dataworkshop/Desktop/Spark/FG_PARAM.xlsx"

# Read the Excel file into a Pandas DataFrame
FG_Param = pd.read_excel(excel_file_path)

# Convert the Pandas DataFrame to a PySpark DataFrame
FG_Param = spark.createDataFrame(FG_Param)

FG_Param = FG_Param.withColumnRenamed("AAP", "_AAP")


# REPORTING

file_path = "/Users/dataworkshop/Desktop/Spark/Table FG paramètrage frais reporting propal 2.xlsx"
AAP = "AAP"

    # transform excel in to dataframe
df_reporting_annees = pd.read_excel(file_path,AAP, header=0)
param_csv =df_reporting_annees.to_csv("df_reporting_annees.csv")
df_reporting_annees = spark.createDataFrame(pd.read_csv('df_reporting_annees.csv').replace({float('nan'): None}))

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [4]:
df.columns

['Dispositif',
 'AAP',
 'Date_de_releve',
 'Acronyme_du_projet',
 'Nom_de_l_entreprise',
 'Chef_de_file',
 'Assiette',
 'Statut',
 'Nb_de_partenaires',
 'Thematique',
 'Type_d_entreprise',
 'Code_postal_de_realisation_partenaire',
 'Ville_de_realisation_partenaire',
 'Pole_de_competitivite_labellisateur',
 'Objet_du_projet',
 'Retombees_economiques',
 'Direction_Regionale_Bpifrance',
 'SIRET',
 'SIREN',
 'N_projet',
 'Partenaire_Nom_du_responsable_legal',
 'Partenaire_Prenom_du_responsable_legal',
 'Partenaire_Civilite_du_responsable_legal',
 'Partenaire_Adresse_email_du_responsable_legal',
 'Partenaire_Nom_du_responsable_projet',
 'Partenaire_Prenom_du_responsable_projet',
 'Partenaire_Civilite_du_responsable_projet',
 'Partenaire_Adresse_email_du_responsable_projet',
 'Date_de_verrouillage',
 'Commentaire_de_verrouillage',
 'Personne_affectee_Eligibilite_LCBFT_OAD',
 'Dossier_complet',
 'Societe_Immatriculee_en_France',
 'A_jour_obligations_sociales',
 'A_jour_obligations_fiscales',


# F_G_AAP_Phase_Cadrage

In [5]:
def F_G_AAP_Phase_Cadrage(df, PARAM_FG, FG_Param, df_reporting_annees):
    # Les données à ramener du dataset
    import pandas as pd
    from pyspark import SparkContext
    from pyspark.sql import SQLContext
    from pyspark.sql.functions import col,lpad, length, lit, to_date,to_timestamp, when, date_format, coalesce, md5, sha2, concat_ws,trim
    from pyspark.sql import functions as F

    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, when
    import datetime
    from pyspark.sql.functions import round

    F_G_AAP_Phase_Cadrage = df.select("AAP", "Dispositif_de_reference", "Date_de_releve","Identifiant_projet")

    # Adjust AAP Nuclaire and all AAP's

    #delarer une vairable de table de param
    aap_mapping_value = PARAM_FG[["Identifiant projet","AAP_Gré-à-gré"]]

    # rename ID_projet to get rid of ambiguity
    aap_mapping_value = aap_mapping_value.withColumnRenamed("Identifiant projet", "Identifiant projet_")



    # Alias the DataFrames
    df1 = F_G_AAP_Phase_Cadrage.alias("df1")
    df2 = aap_mapping_value.alias("df2")

    # Join with alias and specify the columns with qualified names
    F_G_AAP_Phase_Cadrage = df1.join(df2, df1["Identifiant_projet"] == df2["Identifiant projet_"], "left_outer")

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
        "AAP",
        when(col("AAP").startswith("Gré-à-gré Etat"), col("AAP_Gré-à-gré"))
        .otherwise(F_G_AAP_Phase_Cadrage["AAP"]))

    # supprimer les deux colonnes ID_projet
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet")
    #F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet_")



    # Renommer date de relève

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumnRenamed("Date_de_releve", "1ere année de relève")


    # Mapper Catégorie AAP

    FG_Param_ =FG_Param[["_AAP","Catégorie_AAP"]]

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(FG_Param_, F_G_AAP_Phase_Cadrage["AAP"] == FG_Param_["_AAP"], "left_outer")
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("_AAP")

    # Mapper convention 

    FG_Param1 =FG_Param[["_AAP","Convention_AAP"]]

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(FG_Param1, F_G_AAP_Phase_Cadrage["AAP"] == FG_Param1["_AAP"], "left_outer")
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("_AAP")
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumnRenamed("Convention_AAP", "Convention")


    # Col Cadrer la procedure - AAP renouvelé

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", when((F_G_AAP_Phase_Cadrage["AAP"].substr(-1, 1) != "1") & (F_G_AAP_Phase_Cadrage["AAP"].substr(-1, 1) != "2"), 5500))
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", col("Cadrer la procédure - AAP renouvelé").cast("int"))

    # Col Cadrer la procedure 

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", when(F_G_AAP_Phase_Cadrage["Cadrer la procédure - AAP renouvelé"].isNull(), lit(11167)))
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", col("Cadrer la procédure").cast("int"))

    # les exceptions de gré-à-gré

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn('Cadrer la procédure', when(col('Identifiant projet_').isNotNull(), 33583).otherwise(col('Cadrer la procédure')))

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet_")
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("AAP_Gré-à-gré")

    # get distinct
    F_G_AAP_Phase_Cadrage.distinct()

    # plusieurs colonnes de Reporrting & Evaluation 

    # ramener le montant qu'on va diviser
    MT_Reporting_evaluation = PARAM_FG.filter(col("Reporting et évaluation HT").isNotNull())
    MT_Reporting_evaluation = MT_Reporting_evaluation[["Reporting et évaluation HT"]]
    MT_Reporting_evaluation = MT_Reporting_evaluation.withColumn("Reporting et évaluation HT", col("Reporting et évaluation HT").cast("int"))

    # ramner les données de parametrage pour identifier les AAP sur les quel on va diviser le montant de reporting evaluation



    # Assuming you have a SparkSession named 'spark'

    def calculate_result_for_year(df_reporting_annees, MT_Reporting_evaluation, year):
        # Filter rows where the column for the specific year is 1
        df_reporting_annees_year = df_reporting_annees.filter(col(f'AAP Actifs {year}') == 1)

        # Count the occurrences
        count_of_ones = df_reporting_annees_year.select('AAP').distinct().count()

        # Get the integer value from the second table
        reporting_evaluation_value = MT_Reporting_evaluation.select("Reporting et évaluation HT").collect()[0][0]

        # Divide the integer value by the total number of active AAPs for the specific year
        result = reporting_evaluation_value / count_of_ones

        # Update the DataFrame with the calculated result for the specific year
        df_reporting_annees = df_reporting_annees.withColumn(f'AAP Actifs {year}', when(col(f'AAP Actifs {year}') == 1, result).otherwise(None))

        return df_reporting_annees

    # Get the current year
    current_year = datetime.datetime.now().year

    # Calculate results for each year from 2021 to the current year
    for year in range(2021, current_year + 1):
        df_reporting_annees = calculate_result_for_year(df_reporting_annees, MT_Reporting_evaluation, year)

    columns_to_drop = ["Unnamed: 0","Catégorie_AAP", "Convention_AAP"]
    df_reporting_annees = df_reporting_annees.drop(*columns_to_drop)


    # arrandire les AAP actifs

    # Voici comment tu peux arrondir toutes ces colonnes
    colonnes_a_arrondir = ["AAP Actifs 2022", "AAP Actifs 2023", "AAP Actifs 2024", "AAP Actifs 2025", "AAP Actifs 2026",
                           "AAP Actifs 2027", "AAP Actifs 2028", "AAP Actifs 2029", "AAP Actifs 2030", "AAP Actifs 2031",
                           "AAP Actifs 2032", "AAP Actifs 2033", "AAP Actifs 2034", "AAP Actifs 2035"]

    # Appliquer la fonction round à chaque colonne
    for colonne in colonnes_a_arrondir:
        df_reporting_annees = df_reporting_annees.withColumn(colonne, round(colonne, 0))

    # joindre la premiere et la deuximee parite
    df_reporting_annees = df_reporting_annees.withColumnRenamed("AAP", "AAP_")

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(df_reporting_annees, F_G_AAP_Phase_Cadrage["AAP"] == df_reporting_annees["AAP_"], "left_outer")

    df_reporting_annees = df_reporting_annees.drop("_AAP")



    from pyspark.sql.functions import col, sum, min, max, first

    # convert columns to int


    # Display the similar version of 

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.groupBy("AAP").agg(
        first("Dispositif_de_reference").alias("Dispositif"),
        first("Catégorie_AAP").alias("Catégorie_AAP"),
        first("Convention").alias("Convention"),
        min("1ere année de relève").alias("1ere année de relève"),
        max("Cadrer la procédure").alias("Cadrer la procédure"),
        max("Cadrer la procédure - AAP renouvelé").alias("Cadrer la procédure - AAP renouvelé"),
        first("AAP Actifs 2021").alias("Reporting & Evaluation 2021"),
        first("AAP Actifs 2022").alias("Reporting & Evaluation 2022"),
        first("AAP Actifs 2023").alias("Reporting & Evaluation 2023"),
        min("AAP Actifs 2024").alias("Reporting & Evaluation 2024"),
        min("AAP Actifs 2025").alias("Reporting & Evaluation 2025"),
        min("AAP Actifs 2026").alias("Reporting & Evaluation 2026"),
        min("AAP Actifs 2027").alias("Reporting & Evaluation 2027"),
        min("AAP Actifs 2028").alias("Reporting & Evaluation 2028"),
        min("AAP Actifs 2029").alias("Reporting & Evaluation 2029"),
        min("AAP Actifs 2030").alias("Reporting & Evaluation 2030"),
        min("AAP Actifs 2031").alias("Reporting & Evaluation 2031"),
        min("AAP Actifs 2032").alias("Reporting & Evaluation 2032"),
        min("AAP Actifs 2033").alias("Reporting & Evaluation 2033"),
        min("AAP Actifs 2034").alias("Reporting & Evaluation 2034"),
        min("AAP Actifs 2035").alias("Reporting & Evaluation 2035"),
    )

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation 2021", col("Reporting & Evaluation 2021").cast("int"))
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation 2022", col("Reporting & Evaluation 2022").cast("int"))
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation 2023", col("Reporting & Evaluation 2023").cast("int"))

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", col("Cadrer la procédure").cast("int"))
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", col("Cadrer la procédure - AAP renouvelé").cast("int"))



    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
        "Reporting & Evaluation TOTAL",
        F.col("Reporting & Evaluation 2021") + F.col("Reporting & Evaluation 2022") + F.col("Reporting & Evaluation 2023")
    )

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation TOTAL", col("Reporting & Evaluation TOTAL").cast("int"))


    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
        "Total frais de gestion par AAP",
        F.coalesce(
            F.col("Reporting & Evaluation TOTAL"), 
            F.lit(0)
        ) + F.coalesce(
            F.col("Cadrer la procédure"), 
            F.lit(0)
        ) + F.coalesce(
            F.col("Cadrer la procédure - AAP renouvelé"), 
            F.lit(0)
        )
    )

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("AAP_peaufiner", col("AAP").alias("AAP_peaufiner"))

    # get  Similar gré-àgré

    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("AAP", when(col("AAP").startswith("Gré-à-gré Etat"), "Gré-à-gré Etat").otherwise(col("AAP")))
    F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", when(F_G_AAP_Phase_Cadrage["AAP"] == "Gré-à-gré Etat", None).otherwise(F_G_AAP_Phase_Cadrage["Cadrer la procédure - AAP renouvelé"]))
    return F_G_AAP_Phase_Cadrage

# Show the result


In [6]:
FDG_AAP_Phase_Cadrage_Final = F_G_AAP_Phase_Cadrage(df, PARAM_FG, FG_Param, df_reporting_annees)
FDG_AAP_Phase_Cadrage_Final.show(1000,False)

24/01/10 09:13:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------------+----------+---------------+--------------------+--------------------+-------------------+-----------------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+------------------------------+
|AAP                    |Dispositif|Catégorie_AAP  |Convention          |1ere année de relève|Cadrer la procédure|Cadrer la procédure - AAP renouvelé|Reporting & Evaluation 2021|Reporting & Evaluation 2022|Reporting & Evaluation 2023|Reporting & Evaluation 2024|Reporting & Evaluation 2025|Reporting & Evaluation 2026|Reporting & Evaluation 2027|Reporting & Evaluation 

# F_G_Projet_Phase_1

ici les deux df pour joindre phase_cadrage et df

FDG_AAP_Phase_Cadrage_Final[["AAP","Dispositif","AAP","Catégorie_AAP","Convention"]] 

F_G_projet_1 = df[["Identifiant_projet",'Acronyme_du_projet', 'Instruction_affectee_autre_operateur', 
                 'COPIL_Date_de_signature_DPM',"Montant_total_de_l_aide_retenue_HT", 'Date_de_verrouillage',"COPIL_Decision_Comite"]]


In [7]:

F_G_projet_1 = df[["Identifiant_projet", 'Dispositif_de_reference',"AAP", 'Acronyme_du_projet', 'Instruction_affectee_autre_operateur', 
                 'COPIL_Date_de_signature_DPM',"Montant_total_de_l_aide_retenue_HT", 'Date_de_verrouillage',"COPIL_Decision_Comite"]]

F_G_projet_1 = F_G_projet_1.withColumnRenamed("Identifiant_projet", "Identifiant projet")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Dispositif_de_reference", "Dispositif")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Acronyme_du_projet", "Acronyme du projet")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Instruction_affectee_autre_operateur", "Instruction affectée autre opérateur")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Catégorie_AAP", "Catégorie Dispositif")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("COPIL_Decision_Comite", "COPIL : Décision Comité")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Date_de_verrouillage", "Date de verrouillage")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("COPIL_Date_de_signature_DPM", "COPIL : Date de signature DPM")


F_G_projet_1 = F_G_projet_1.withColumn("Montant total de l'aide retenue HT", col("Montant_total_de_l_aide_retenue_HT").cast("int"))


#-------------------------------------------------------------------------------------------------------------------------------------------------------
# renommé 'Dispositif / Opérateur cible' ==> Dispositif de sortie
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Dispositif / Opérateur cible", "Dispositif de sortie")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Catégorie Dispositif  ==> Convention AAP dans F_G_AAP_Phase_Cadrage

FG_Param1 =FG_Param[["_AAP","Convention_AAP"]]
F_G_projet_1 = F_G_projet_1.join(FG_Param1, F_G_projet_1["AAP"] == FG_Param1["_AAP"], "left_outer")
F_G_projet_1 = F_G_projet_1.drop("_AAP")

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Convention AAP ==> convention dans F_G_AAP_Phase_Cadrage
FG_Param2 =FG_Param[["_AAP","Catégorie_AAP"]]
F_G_projet_1 = F_G_projet_1.join(FG_Param2, F_G_projet_1["AAP"] == FG_Param2["_AAP"], "left_outer")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Catégorie_AAP", "Catégorie Dispositif")
F_G_projet_1 = F_G_projet_1.drop("_AAP")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# sous-fonds FG == 

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# renommé Instruction affectée autre opérateur ==> Instruction autre opérateur
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Instruction affectée autre opérateur", "Instruction autre opérateur")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de projet Décidés COPIL ==> regarder la RG dans neurone   " Une histoire de date de cloture "

condition_nb_projet_decide = col("COPIL : Décision Comité") == 'Favorable'
F_G_projet_1 = F_G_projet_1.withColumn("Nombre de projet décidés COPIL", when(condition_nb_projet_decide, 1).otherwise(None))
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de partenaire Décidés COPIL ==> regarder la RG dans neurone   " Une histoire de date de cloture "
# cette colonne sera créer dans le groupby

# Assuming F_G_projet_1 is your DataFrame
F_G_Nombre_patrtenaire = F_G_projet_1.groupBy("Identifiant projet").agg(
    F.sum("Nombre de projet décidés COPIL").alias("Nombre de partenaire décidés COPIL")
)
F_G_Nombre_patrtenaire = F_G_Nombre_patrtenaire.withColumnRenamed("Identifiant projet", "Identifiant projet_")

F_G_projet_1 = F_G_projet_1.join(F_G_Nombre_patrtenaire, F_G_projet_1["Identifiant projet"] == F_G_Nombre_patrtenaire["Identifiant projet_"], "left_outer")
F_G_projet_1 = F_G_projet_1.drop("Identifiant projet_")


#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de projets verrouillés : Sum of notnull values in date de verrouillage
confition_nb_pr_verrouillés = col("Date de verrouillage").isNotNull()
F_G_projet_1 = F_G_projet_1.withColumn("Nombre de projets verrouilles", when(confition_nb_pr_verrouillés, 1).otherwise(0))

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Mapper Reveuillir les candidatures deuis Param_FG

receuille_candi =PARAM_FG[["Année","Catégorie","Recueillir les candidatures HT"]]

#format année
receuille_candi = receuille_candi.withColumn('Année', receuille_candi['Année'].cast(IntegerType()))
receuille_candi = receuille_candi.withColumn('Année', receuille_candi['Année'].cast(StringType()))

# Receuillir_key
receuille_candi = receuille_candi.withColumn("Receuillir_key", concat_ws("_", col("Année"), col("Catégorie"))) 
#receuille_candi = receuille_candi.drop("Année")
#receuille_candi = receuille_candi.drop("Catégorie")


# create year 
F_G_projet_1 = F_G_projet_1.withColumn("Date de verrouillage", F.to_date(col("Date de verrouillage")))
F_G_projet_1 = F_G_projet_1.withColumn("Année de verrouillage" , year(F_G_projet_1["Date de verrouillage"]))

F_G_projet_1 = F_G_projet_1.withColumn("Receuillir_key_", concat_ws("_", col("Année de verrouillage"), col("Catégorie Dispositif"))) 


F_G_projet_1 = F_G_projet_1.join(receuille_candi, F_G_projet_1["Receuillir_key_"] == receuille_candi["Receuillir_key"], "left_outer") 

F_G_projet_1 = F_G_projet_1.drop("Receuillir_key_")
F_G_projet_1 = F_G_projet_1.drop("Receuillir_key")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------

# Display the similar version of 

F_G_projet_final = F_G_projet_1.groupBy("Identifiant projet").agg(
    first("Dispositif").alias("Dispositif de sortie"),
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention_AAP").alias("Convention AAP"),
    first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    max("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Date de verrouillage").alias("Date de verrouillage"),    
    first("Nombre de projets verrouilles").alias("Nombre de projets verrouilles"),
    max("Recueillir les candidatures HT").alias("Recueillir les candidatures HT"),
)


In [8]:
F_G_projet_final.show()

+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------------------+-----------------------------+----------------------------------+------------------------------+----------------------------------+--------------------+-----------------------------+------------------------------+
|  Identifiant projet|Dispositif de sortie|                 AAP|Acronyme du projet|Catégorie Dispositif|      Convention AAP|Instruction autre opérateur|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Date de verrouillage|Nombre de projets verrouilles|Recueillir les candidatures HT|
+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------------------+-----------------------------+----------------------------------+------------------------------+-----------------

24/01/10 09:13:27 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
